In [6]:
%%time

# load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings  # For handling warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

print("Done!")

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Done!
CPU times: user 1.66 s, sys: 130 ms, total: 1.79 s
Wall time: 2.53 s


In [7]:
# locate data
import os
os.listdir('../aru-hackathon-team1')

['coding_space.ipynb',
 'corona_tested_individuals_ver_006.english.csv',
 'README.md',
 '.git']

In [8]:
# load data
df = pd.read_csv("final_df.csv")
print(df.shape)
df.head(5)

(274702, 14)


,days_since_start,cough,fever,sore_throat,shortness_of_breath,head_ache,is_corona_positive,test_indication_Abroad,test_indication_Contact with confirmed,test_indication_Other,age_60_and_above_NaN,age_60_and_above_Yes,gender_NaN,gender_male
0,50,0,0,0,0,0,0,0,0,1,1,0,0,0
1,50,1,0,0,0,0,0,0,0,1,1,0,0,0
2,50,0,1,0,0,0,0,0,0,1,1,0,0,1
3,50,1,0,0,0,0,0,0,0,1,1,0,0,0
4,50,1,0,0,0,0,0,0,0,1,1,0,0,1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTEENN

In [12]:
# In the first step we will split the data in training and remaining dataset
X_train, X_rem, y_train, y_rem = train_test_split(df.drop('is_corona_positive', axis=1), df['is_corona_positive'], test_size=0.2, random_state=42)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [13]:
X_smote_train, y_smote_train = SMOTEENN(random_state=42).fit_resample(X_train, y_train)

In [15]:
from collections import Counter
print(Counter(y_smote_train))

Counter({0: 194788, 1: 78825})


In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_smote_train)
X_valid_scaled, X_test_scaled = scaler.transform(X_valid), scaler.transform(X_test)

In [27]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_valid_scaled = pd.DataFrame(X_valid_scaled, columns=X_valid.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

y_smote_train = pd.Series(y_smote_train)
y_valid = pd.Series(y_valid)
y_test = pd.Series(y_test)

train_df = pd.concat([X_train_scaled, y_smote_train], axis=1)
valid_df = pd.concat([X_valid_scaled, y_valid], axis=1)
test_df = pd.concat([X_test_scaled, y_test], axis=1)


In [28]:
display(train_df.head(5))

,days_since_start,cough,fever,sore_throat,shortness_of_breath,head_ache,test_indication_Abroad,test_indication_Contact with confirmed,test_indication_Other,age_60_and_above_NaN,age_60_and_above_Yes,gender_NaN,gender_male,is_corona_positive
0,-1.209602,-0.512428,-0.438907,-0.223578,-0.18135,-0.30037,-0.300897,-0.515508,0.647058,-0.82621,-0.3388,-0.276227,-0.903927,0
1,-0.641272,-0.512428,-0.438907,-0.223578,-0.18135,-0.30037,-0.300897,-0.515508,0.647058,-0.82621,-0.3388,3.620213,-0.903927,0
2,-0.154131,-0.512428,-0.438907,-0.223578,-0.18135,-0.30037,-0.300897,-0.515508,0.647058,-0.82621,-0.3388,-0.276227,-0.903927,0
3,-0.235321,-0.512428,-0.438907,-0.223578,-0.18135,-0.30037,-0.300897,-0.515508,0.647058,-0.82621,-0.3388,-0.276227,-0.903927,0
4,-0.478891,-0.512428,-0.438907,-0.223578,-0.18135,-0.30037,-0.300897,-0.515508,0.647058,-0.82621,-0.3388,-0.276227,1.106283,0


In [29]:
train_df.to_csv("train_df.csv", index=False)
valid_df.to_csv("valid_df.csv", index=False)
test_df.to_csv("test_df.csv", index=False)